In [ ]:
import sys
sys.path.insert(0, "/home/m_evgeniy/DeepLearning/OrangePi")

from orangepi.imports import *
from orangepi.__init__ import *
from orangepi.dataset_utils import *
from orangepi.model_utils import *

# Выбираем модель

## Segformer-B0

In [ ]:
transform = None
processor = SegformerImageProcessor.from_pretrained(
    "nvidia/segformer-b0-finetuned-ade-512-512"
)
model = SegformerForSemanticSegmentation.from_pretrained(
    "nvidia/segformer-b0-finetuned-ade-512-512",
    num_labels=NUM_CLASSES,  # Меняем количество выходных классов
    ignore_mismatched_sizes=True,  # Позволяет менять последний слой
).to(DEVICE)

model.decode_head.classifier = nn.Conv2d(256, NUM_CLASSES, kernel_size=1)
torch.nn.init.xavier_uniform_(model.decode_head.classifier.weight)
torch.nn.init.zeros_(model.decode_head.classifier.bias)

model_type = "segformer"

## Deeplab v3

In [ ]:
processor = None
transform = T.Compose([T.Resize((1024, 1024)), T.ToTensor()])

model = deeplab_model()
model.classifier[4] = nn.Conv2d(256, NUM_CLASSES, kernel_size=1)
model = model.to(DEVICE)

model_type = "deeplab"

In [ ]:
# смотрим параметры модели
summary(
    model=model,
    input_size=(1, 3, 1024, 1024),
    col_names=["input_size", "output_size", "num_params", "trainable"],
    col_width=20,
    row_settings=["var_names"],
)

# Инициализируем датасет

In [ ]:
dataset_path = Path("../data/DeepGlobeDataset/images_and_masks")
# создаем папки train, valid и test (расскоментировать при необходимости)
# split_dataset(dataset_path)

In [ ]:
train_dir = dataset_path / "train"
valid_dir = dataset_path / "valid"
test_dir = dataset_path / "test"

train_dataset = DeepGlobeDataset(
    train_dir, model_type, transform=transform, processor=processor, image_size=1024
)
valid_dataset = DeepGlobeDataset(
    valid_dir, model_type, transform=transform, processor=processor, image_size=1024
)
test_dataset = DeepGlobeDataset(
    test_dir, model_type, transform=transform, processor=processor, image_size=1024
)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=8, shuffle=True)
test_loader  =  DataLoader(test_dataset, batch_size=8, shuffle=True)
len(train_loader), len(valid_loader), len(test_loader)

### Смотрим примеры картинок с которыми работаем

In [ ]:
data = next(iter(train_loader))
num_examples = 5

fig, axes = plt.subplots(2, num_examples, figsize=(12, 8))

images, masks = data
for i in range(num_examples):
    axes[0, i].imshow(images[i].permute(1, 2, 0))
    axes[0, i].set_title("Image")
    axes[1, i].imshow(masks[i])
    axes[1, i].set_title("Mask")

# Обучение

In [ ]:
epochs = 10
lr = 1e-3

# Запускаем обучение
model_save_name = "DeeplabV3_DeepGlobe_V1"
assert model_save_name is not None, "Enter model's name for saving"

criterion = nn.CrossEntropyLoss()
best_model, loss_history, iou_score_history, dice_score_history = train(
    model,
    train_loader,
    valid_loader,
    criterion,
    lr,
    epochs,
    model_type,
    verbose=True,
    is_scheduler=False,
)

# --- 6. Сохранение модели ---
torch.save(best_model.state_dict(), f"../models/{model_save_name}.pth")
print("Модель сохранена!")

In [ ]:
visualize_losses_and_scores(loss_history, iou_score_history, dice_score_history)